Code to reproduce running example of

B. Legat, R. M. Jungers, and P. A. Parrilo
[**Certifying unstability of Switched Systems using Sum of Squares Programming**](https://arxiv.org/abs/1710.01814)

In [ ]:
Pkg.clone("https://github.com/blegat/HybridSystems.jl")
Pkg.clone("https://github.com/blegat/SwitchOnSafety.jl")

In [ ]:
include(Pkg.dir("SwitchOnSafety", "test", "solvers.jl"))
@assert !isempty(sdp_solvers)
solver = first(sdp_solvers)

In [1]:
import Pkg
Pkg.develop(Pkg.PackageSpec(path="../../SumOfSquares"))
Pkg.develop(Pkg.PackageSpec(path="../../SetProg"))

 Resolving package versions...
  Updating `~/.julia/dev/SwitchOnSafety/Project.toml`
 [no changes]
  Updating `~/.julia/dev/SwitchOnSafety/Manifest.toml`
 [no changes]
 Resolving package versions...


┌ Info: Assigning UUID fc078b4a-c1ae-5945-9d19-22107d1bf3e1 to SetProg
└ @ Pkg.Types /build/julia/src/julia-1.1.0/usr/share/julia/stdlib/v1.1/Pkg/src/Types.jl:841


  Updating `~/.julia/dev/SwitchOnSafety/Project.toml`
 [no changes]
  Updating `~/.julia/dev/SwitchOnSafety/Manifest.toml`
 [no changes]


In [2]:
import Pkg
Pkg.status()

Project SwitchOnSafety v0.0.0
    Status `~/.julia/dev/SwitchOnSafety/Project.toml`
  [861a8166] + Combinatorics v0.7.0
  [7c1d4256] + DynamicPolynomials v0.2.0
  [1a297f60] + FillArrays v0.5.0
  [a2cc645c] + GraphPlot v0.3.1
  [2207ec0c] + HybridSystems v0.1.5
  [4076af6c] + JuMP v0.19.0
  [093fc24a] + LightGraphs v1.2.0
  [b8f27783] + MathOptInterface v0.8.4
  [d14a8603] + MathematicalSystems v0.6.4
  [f4abf1af] + MultivariateMoments v0.2.1
  [102ac46a] + MultivariatePolynomials v0.2.8
  [429524aa] + Optim v0.18.1
  [774612a8] + ParameterJuMP v0.1.0
  [67491407] + Polyhedra v0.5.0
  [c46f51b8] + ProfileView v0.4.0
  [3cdcf5f2] + RecipesBase v0.6.0
  [189a3867] + Reexport v0.2.0
  [8e049039] + SemialgebraicSets v0.1.0
  [fc078b4a] + SetProg v0.0.0 [`../SetProg`]
  [90137ffa] + StaticArrays v0.10.3
  [4b9e565b] + SumOfSquares v0.3.1 [`../SumOfSquares`]
  [37e2e46d] + LinearAlgebra 
  [2f01184e] + SparseArrays 
  [10745b16] + Statistics 


In [3]:
using HybridSystems
include(joinpath(dirname(dirname(pathof(HybridSystems))), "examples", "PEDJ16s4.jl"))
using SwitchOnSafety

┌ Info: Recompiling stale cache file /home/blegat/.julia/compiled/v1.1/SwitchOnSafety/EUPLd.ji for SwitchOnSafety [ceb7f16a-07bf-5f4a-9354-b68f01b1610f]
└ @ Base loading.jl:1184


As showed in [Example 7, LJP16], the JSR is 0.9748171979372074 which is attained for the cycle
$$(3, 1, 2), (1, 3, 1), (3, 1, 2), (1, 2, 3), (2, 3, 1), (3, 3, 1)^3$$
which corresponds to the sequence of symbols: 21231111.

[LJP16] B. Legat, R. M. Jungers and P. A. Parrilo,
*Generating unstable trajectories for Switched Systems via Dual Sum-Of-Squares techniques.*
Proceedings of the 19th International Conference on Hybrid Systems: Computation and Control,
ACM, **2016**, 51-60.

In [23]:
function t(pair::Pair)
    edge = HybridSystems.edge_object(hs.automaton, pair.first, pair.second)
    # As there is no multiple edges, i.e. edges with same source and target but different label,
    # we can get the transition as follows
    @assert isone(length(hs.automaton.Σ[edge]))
    id = first(keys(hs.automaton.Σ[edge]))
    return HybridSystems.LightTransition(edge, id)
end
smp = periodicswitching(hs, t.([3 => 1, 1 => 3, 3 => 1, 1 => 2, 2 => 3, 3 => 3, 3 => 3, 3 => 3]))

PSW(0.9748172, [2, 1, 2, 3, 1, 1, 1, 1])

The matrix 1 can be repeated since it is part of the self-loop (3, 3, 1) so it also provides a lower bound but it is not an s.m.p.

In [24]:
psw1 = periodicswitching(hs, t.([3 => 3]))

PSW(0.93925502, [1])

The following periodic switchings have a high growth rates but are not s.m.p.

In [25]:
psw2 = periodicswitching(hs, t.([3 => 1, 1 => 3, 3 => 2, 2 => 3, 3 => 3, 3 => 3, 3 => 3]))

PSW(0.96592844, [2, 1, 3, 1, 1, 1, 1])

In [26]:
psw3 = periodicswitching(hs, t.([3 => 1, 1 => 3, 3 => 1, 1 => 3, 3 => 3, 3 => 3, 3 => 3, 3 => 3]))

PSW(0.97289401, [2, 1, 2, 1, 1, 1, 1, 1])

## Gripenberg

With a maximum length of 6, the Gripenberg algorithm finds `psw1`.

In [20]:
@time psw, ub = gripenberg(hs, max_length = 6)

ρ evaluations   : 425 < 1000 = max_ρ_eval.
norm evaluations: 1567 < 10000 = max_norm_eval.
switch length   : 6 ≥ 6 = max_length.
  0.011959 seconds (80.99 k allocations: 5.459 MiB)


(PSW(0.93925502, [1, 1, 1, 1, 1, 1]), 1.039593578184173)

With a maximum length of 7, the Gripenberg algorithm finds `psw2`.

In [21]:
@time psw, ub = gripenberg(hs, max_length = 7)

ρ evaluations   : 834 < 1000 = max_ρ_eval.
norm evaluations: 3137 < 10000 = max_norm_eval.
switch length   : 7 ≥ 7 = max_length.
  0.022956 seconds (160.82 k allocations: 10.914 MiB)


(PSW(0.96592844, [1, 1, 1, 2, 1, 3, 1]), 1.0181518243613532)

In [14]:
@time psw, ub = gripenberg(hs, max_ρ_eval = 2000)

ρ evaluations   : 1196 < 2000 = max_ρ_eval.
norm evaluations: 4544 < 10000 = max_norm_eval.
switch length   : 9 < 50 = max_length.
  0.043515 seconds (231.92 k allocations: 15.806 MiB, 20.31% gc time)


(PSW(0.9748172, [1, 1, 1, 2, 1, 2, 3, 1]), 0.9848171979372075)

In [16]:
@time psw, ub = gripenberg(hs, max_length = 8)

ρ evaluations   : 1134 ≥ 1000 = max_ρ_eval.
norm evaluations: 4296 < 10000 = max_norm_eval.
switch length   : 8 ≥ 8 = max_length.
  0.032864 seconds (219.44 k allocations: 14.940 MiB)


(PSW(0.9748172, [1, 1, 1, 2, 1, 2, 3, 1]), 1.0011588099570934)

ρ evaluations   : 834 < 1000 = max_ρ_eval.
norm evaluations: 3137 < 10000 = max_norm_eval.
switch length   : 7 ≥ 7 = max_length.
  0.029557 seconds (160.82 k allocations: 10.914 MiB, 23.02% gc time)


(PSW(0.96592844, [1, 1, 1, 2, 1, 3, 1]), 1.0181518243613532)

ρ evaluations   : 425 < 1000 = max_ρ_eval.
norm evaluations: 1567 < 10000 = max_norm_eval.
switch length   : 6 ≥ 6 = max_length.
  0.018538 seconds (80.99 k allocations: 5.459 MiB, 33.67% gc time)


(PSW(0.93925502, [1, 1, 1, 1, 1, 1]), 1.039593578184173)

## SumOfSquares

In [4]:
using MosekTools
using JuMP
factory = with_optimizer(Mosek.Optimizer, QUIET=true);

## Example 3.6

In [9]:
using MultivariatePolynomials
function plot_lyap(p)
    d = mindegree(p)
    @assert d == maxdegree(p)
    @assert nvariables(p) == 2
    α = linspace(0., 2π, 1000)
    X = cos.(α)
    Y = sin.(α)
    λ = map(x -> p(variables(p) => x), zip(X, Y))
    λ = λ.^(1/d)
    plot(X ./ λ, Y ./ λ, aspect_ratio=:equal, label="", grid=false, border=nothing, ticks=nothing, linewidth=3)#, grid=false, label="")
    plot!(X, Y, label="", linestyle=:dash, linecolor=:black, linewidth=2)
end

plot_lyap (generic function with 1 method)

In [ ]:
gr()
for d in 1:6
    for v in 1:4
        plot_lyap(get(s.lyaps[d]).primal[v])
        savefig("PEDJ_d" * string(d) * "_v" * string(v) * ".png")
    end
end

## Example 4.3

In [14]:
using LinearAlgebra
using MultivariatePolynomials
using MultivariateMoments
d = 1
tol = 1e-4
lyap = getlyap(hs, d; factory=factory, tol=tol)
for s in states(hs)
    for t in out_transitions(hs, s)
        σ = symbol(hs, t)
        ν = lyap.dual[t]
        a = extractatoms(ν, tol)
        if a !== nothing
            v = target(hs, t)
            atom = a.atoms[1]
            x = atom.center
            nx = norm(x, 2)
            y = round.(x / nx, digits=3)
            println("μ_{$s$v$σ} = $nx * δ_{$y}")
            @show normalize(SwitchOnSafety.dynamicforσ(hs, σ) * y)
        end
    end
end

μ_{131} = 1.0907033942729127 * δ_{[0.917, 0.399]}
normalize(SwitchOnSafety.dynamicforσ(hs, σ) * y) = [0.999626, -0.027334]
μ_{312} = 1.143222101937864 * δ_{[0.875, 0.485]}
normalize(SwitchOnSafety.dynamicforσ(hs, σ) * y) = [0.916808, 0.399328]
μ_{331} = 1.3210537518102323 * δ_{[0.757, -0.653]}
normalize(SwitchOnSafety.dynamicforσ(hs, σ) * y) = [0.422686, -0.906276]


## Example 4.10

In [38]:
function sequencelb(s, d, l, p_0s = [:Primal, :Random], v_0s = states(s))
    best = 0.
    for p_0 in p_0s
        for v_0 in v_0s
            println("d = $d, l = $l, p_0 = $p_0, v_0 = $v_0")
            println("Build sequence")
            @time seq = sosbuildsequence(s, d, p_0=p_0, v_0=v_0, l=l, niter=100)
            println("Find smp")
            @time psw = findsmp(seq)
            @show psw
            @assert psw !== nothing
            best = max(best, psw.growthrate)
        end
    end
    return best
end

sequencelb (generic function with 3 methods)

In [34]:
soslb = zeros(6)
sosub = zeros(6)
sosex = zeros(6)
seql = [zeros(6) for i in 1:3];

In [35]:
function full_sos(d; ls=1:3, tol=5e-4, reset=false)
    sosdata(hs).lb = 0.
    sosdata(hs).smp = nothing
    if reset
        sosdata(hs).lyaps[d] = nothing
    end
    @time soslb[d], sosub[d] = soslyapb(hs, d, factory=factory, tol=tol, ranktols=[], disttols=[])
    if hassmp(hs)
        sosex[d] = getsmp(hs).growthrate
    end
    for l in ls
        @time seql[l][d] = sequencelb(hs, d, 1)
    end
end

full_sos (generic function with 2 methods)

In [39]:
full_sos(1, reset=true)

  1.577425 seconds (626.39 k allocations: 30.855 MiB, 0.49% gc time)
d = 1, l = 1, p_0 = Primal, v_0 = 1
Build sequence
  0.014735 seconds (35.40 k allocations: 2.133 MiB, 77.77% gc time)
Find smp
  0.019546 seconds (203.08 k allocations: 16.201 MiB)
psw = PSW(0.97289401, [1, 1, 1, 1, 1, 2, 1, 2])
d = 1, l = 1, p_0 = Primal, v_0 = 2
Build sequence
  0.002807 seconds (35.37 k allocations: 2.131 MiB)
Find smp
  0.030131 seconds (200.56 k allocations: 15.993 MiB, 29.94% gc time)
psw = PSW(0.97289401, [1, 1, 1, 1, 1, 2, 1, 2])
d = 1, l = 1, p_0 = Primal, v_0 = 3
Build sequence
  0.002072 seconds (35.40 k allocations: 2.133 MiB)
Find smp
  0.026911 seconds (205.99 k allocations: 16.450 MiB, 28.66% gc time)
psw = PSW(0.97289401, [1, 1, 1, 1, 2, 1, 2, 1])
d = 1, l = 1, p_0 = Primal, v_0 = 4
Build sequence
  0.002119 seconds (35.35 k allocations: 2.130 MiB)
Find smp
  0.024552 seconds (200.52 k allocations: 16.009 MiB, 28.29% gc time)
psw = PSW(0.97289401, [1, 1, 1, 1, 2, 1, 2, 1])
d = 1, l = 

In [18]:
for d in 1:6
    full_sos(d)
end

  3.246635 seconds (4.99 M allocations: 250.100 MiB, 3.89% gc time)
l = 1
p_0 = :Primal
v_0 = 1
Build sequence
  0.037380 seconds (72.35 k allocations: 4.080 MiB)
Find smp
  0.404629 seconds (1.21 M allocations: 64.452 MiB, 7.91% gc time)
v_0 = 2
Build sequence
  0.002203 seconds (35.37 k allocations: 2.131 MiB)
Find smp
  0.029623 seconds (200.56 k allocations: 15.993 MiB, 31.11% gc time)
v_0 = 3
Build sequence
  0.002062 seconds (35.40 k allocations: 2.133 MiB)
Find smp
  0.026348 seconds (205.99 k allocations: 16.450 MiB, 29.87% gc time)
v_0 = 4
Build sequence
  0.002076 seconds (35.35 k allocations: 2.130 MiB)
Find smp
  0.025045 seconds (200.52 k allocations: 16.009 MiB, 30.52% gc time)
p_0 = :Random
v_0 = 1
Build sequence
  0.002101 seconds (35.41 k allocations: 2.134 MiB)
Find smp
  0.023826 seconds (202.93 k allocations: 16.223 MiB, 27.62% gc time)
v_0 = 2
Build sequence
  0.002268 seconds (35.41 k allocations: 2.134 MiB)
Find smp
  0.026322 seconds (197.60 k allocations: 15.75

  0.031791 seconds (538.10 k allocations: 28.147 MiB, 28.38% gc time)
v_0 = 4
Build sequence
  0.008774 seconds (80.66 k allocations: 5.176 MiB)
Find smp
  0.036043 seconds (230.04 k allocations: 18.310 MiB, 29.76% gc time)
p_0 = :Random
v_0 = 1
Build sequence
  0.005042 seconds (80.75 k allocations: 5.183 MiB)
Find smp
  0.028476 seconds (232.00 k allocations: 18.463 MiB, 27.28% gc time)
v_0 = 2
Build sequence
  0.012606 seconds (80.67 k allocations: 5.177 MiB, 57.91% gc time)
Find smp
  0.019097 seconds (226.14 k allocations: 17.973 MiB)
v_0 = 3
Build sequence
  0.014419 seconds (80.75 k allocations: 5.183 MiB, 55.31% gc time)
Find smp
  0.021388 seconds (232.88 k allocations: 18.504 MiB)
v_0 = 4
Build sequence
  0.016621 seconds (80.71 k allocations: 5.180 MiB, 51.13% gc time)
Find smp
  0.029839 seconds (230.66 k allocations: 18.344 MiB, 25.70% gc time)
  0.324871 seconds (2.81 M allocations: 198.279 MiB, 26.64% gc time)
l = 2
p_0 = :Primal
v_0 = 1
Build sequence
  0.004451 seconds

  0.015801 seconds (143.03 k allocations: 9.802 MiB, 44.80% gc time)
Find smp
  0.013739 seconds (162.74 k allocations: 12.343 MiB)
p_0 = :Random
v_0 = 1
Build sequence
  0.016806 seconds (143.16 k allocations: 9.814 MiB, 45.48% gc time)
Find smp
  0.014634 seconds (164.69 k allocations: 12.484 MiB)
v_0 = 2
Build sequence
  0.016658 seconds (143.02 k allocations: 9.801 MiB, 44.13% gc time)
Find smp
  0.013614 seconds (160.86 k allocations: 12.154 MiB)
v_0 = 3
Build sequence
  0.016349 seconds (143.16 k allocations: 9.814 MiB, 44.98% gc time)
Find smp
  0.013791 seconds (164.90 k allocations: 12.517 MiB)
v_0 = 4
Build sequence
  0.016907 seconds (143.09 k allocations: 9.808 MiB, 44.61% gc time)
Find smp
  0.013784 seconds (162.74 k allocations: 12.343 MiB)
  0.250753 seconds (2.46 M allocations: 178.031 MiB, 23.98% gc time)
l = 2
p_0 = :Primal
v_0 = 1
Build sequence
  0.014786 seconds (106.92 k allocations: 7.575 MiB, 52.09% gc time)
Find smp
  0.014386 seconds (167.79 k allocations: 12

In [ ]:
using Plots
plotlyjs()

In [7]:
font = Plots.Font("sans-serif",12,:hcenter,:vcenter,0.0,RGB{FixedPointNumbers.Normed{UInt8, 8}}(0.0,0.0,0.0))
plot(1:6, xlabel = "d", sosub, label="SOS UB", legendfont=font, linewidth=3)
plot!(1:6, seql3, label="Algorithm 1, l=3", linewidth=3)
plot!(1:6, seql1, label="Algorithm 1, l=1", linewidth=3)
#plot!(1:6, seql2, label="Algorithm 1, l=2")
plot!(1:6, sosex, label="Atom extraction", linewidth=3)
plot!(1:6, soslb, label="SOS LB", linewidth=3)

In [8]:
plot(2:5, xlabel = "d", xticks=2:5, sosub[2:5], label="", linewidth=3)
plot!(2:5, seql3[2:5], label="", linewidth=3)
plot!(2:5, seql1[2:5], label="", linewidth=3)
#plot!(1:6, seql2, label="Algorithm 1, l=2")